# Advanced Querying Mongo

**⚠️ IMPORTANT: Limit your print to avoid infinite scrolling. Otherwise your
code will be lost between print lines. If working with lists do:**

```python
list(collection.find(query))[:5] #or a reasonably low number
```

Importing libraries and setting up connection

In [1]:
# from pymongo import MongoClient
# client = MongoClient("")

from pymongo import MongoClient
import pandas as pd
import time

companies = MongoClient("localhost:27017")
companies

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [2]:
companies.list_database_names()

['admin', 'config', 'ironhack', 'local']

In [3]:
db = companies["ironhack"]
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack')

In [4]:

c = db.get_collection("companies")
c

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack'), 'companies')

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [5]:
query = {"name":"Babelgum"}
projection = {"name":1, "_id":0}

list(c.find(query, projection))


[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [26]:
query = {"number_of_employees":{"$gt":5000}}
projection = {"name":1, "_id":0}

list(c.find(query, projection).sort("number_of_employees",-1).limit(20))



[{'name': 'Siemens'},
 {'name': 'IBM'},
 {'name': 'Toyota'},
 {'name': 'PayPal'},
 {'name': 'Nippon Telegraph and Telephone Corporation'},
 {'name': 'Samsung Electronics'},
 {'name': 'Accenture'},
 {'name': 'Tata Consultancy Services'},
 {'name': 'Flextronics International'},
 {'name': 'Safeway'},
 {'name': 'Sony'},
 {'name': 'LG'},
 {'name': 'Ford'},
 {'name': 'Boeing'},
 {'name': 'Digital Equipment Corporation'},
 {'name': 'Nokia'},
 {'name': 'MItsubishi Electric'},
 {'name': 'MItsubishi Electric'},
 {'name': 'Bertelsmann'},
 {'name': 'Comcast'}]

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [7]:
condition_1 =  {"founded_year":{"$gte":2000}}
condition_2 = {"founded_year":{"$lte":2005}}

query = {"$and": [condition_1, condition_2]}
projection = {"name":1,"founded_year":1, "_id":0}

list(c.find(query, projection))[:5]



[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [27]:
condition_1 =  {"ipo.valuation_amount":{"$gte":100000000}}
condition_2 = {"founded_year":{"$lt":2010}}

query = {"$and": [condition_1, condition_2]}
projection = {"name":1,"ipo":1, "_id":0}

list(c.find(query, projection))[:5]



[{'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'name': 'Yelp',
  'ipo': {'valuation_amount': 1300000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 3,
   'pub_day': 2,
   'stock_symbol': 'NYSE:YELP'}},
 {'name': 'LinkedIn',
  'ipo': {'valuation_amount': 9310000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 2011,
   'pub_month': 7,
   'pub_day': 20,
   'stock_symbol': 'NYSE:LNKD'}},
 {'name': 'Amazon',
  'ipo': {'valuation_amount': 100000000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 1997,
   'pub_month': 5,
   'pub_day': None,
   'stock_symbol': 'NASDAQ:AMZN'}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [9]:
condition_1 =  {"number_of_employees":{"$gt":1000}}
condition_2 = {"founded_year":{"$lt":2005}}

query = {"$and": [condition_1, condition_2]}
projection = {"name":1,"_id":0}

list(c.find(query, projection).sort("number_of_employees", -1).limit(10))






[{'name': 'Siemens'},
 {'name': 'IBM'},
 {'name': 'Toyota'},
 {'name': 'PayPal'},
 {'name': 'Nippon Telegraph and Telephone Corporation'},
 {'name': 'Samsung Electronics'},
 {'name': 'Accenture'},
 {'name': 'Tata Consultancy Services'},
 {'name': 'Flextronics International'},
 {'name': 'Safeway'}]

### 6. All the companies that don't include the `partners` field.

In [10]:
query = {"partners":{"$exists": False}}
projection = {"name":1, "_id":0}

list(c.find(query, projection))

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [28]:
query = {"category_code":{"$type": 10 }}
projection = {"name":1, "_id":0}

list(c.find(query, projection))[:5]




[{'name': 'Collective'},
 {'name': 'Snimmer'},
 {'name': 'KoolIM'},
 {'name': 'Level9 Media'},
 {'name': 'VidKing'}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [29]:
condition_1 =  {"number_of_employees":{"$gt":100}}
condition_2 = {"number_of_employees":{"$lt":1000}}

query = {"$and": [condition_1, condition_2]}
projection = {"name":1, "number_of_employees":1, "_id":0}

list(c.find(query, projection))[:5]

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110}]

### 9. Order all the companies by their IPO price in a descending order.

In [30]:
query = {}
projection = {"name":1,"ipo.valuation_amount":1, "_id":0}

list(c.find(query, projection).sort("ipo.valuation_amount", -1))[:5]

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000.0}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000.0}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000.0}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000.0}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000.0}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [14]:
query={}
projection={"name":1,"number_of_employees":1, "_id":0}

list(c.find(query,projection).sort("number_of_employees", -1).limit(10))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [31]:
condition_1 =  {"founded_month":{"$gt":6}}


query = condition_1
projection = {"name":1,"founded_month":1, "_id":0}

list(c.find(query, projection).limit(1000))[:5]

[{'name': 'Wetpaint', 'founded_month': 10},
 {'name': 'Zoho', 'founded_month': 9},
 {'name': 'Digg', 'founded_month': 10},
 {'name': 'Omnidrive', 'founded_month': 11},
 {'name': 'eBay', 'founded_month': 9}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.000

In [32]:
condition_1 =  {"founded_year":{"$lt":2000}}
condition_2 =  {"acquisition.price_amount":{"$gt":10000000}}

query = {"$and": [condition_1, condition_2]}
projection = {"name":1,"_id":0}

list(c.find(query, projection))[:5]

[{'name': 'Postini'},
 {'name': 'SideStep'},
 {'name': 'Recipezaar'},
 {'name': 'PayPal'},
 {'name': 'Snapfish'}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [33]:
condition_1 = {"acquisition.acquired_year":{"$gt": 2010}}
query = condition_1
projection = {"name":1, "acquisition":1,"_id":0}
list(c.find(query, projection).sort("acquisition.price_amount"))[:5]

[{'name': 'Geni',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2012/11/28/all-in-the-family-myheritage-buys-former-yammer-stablemate-geni-com-raises-25m/',
   'source_description': 'MyHeritage acquires Geni and $25M to build family tree of the whole world',
   'acquired_year': 2012,
   'acquired_month': 11,
   'acquired_day': 28,
   'acquiring_company': {'name': 'MyHeritage', 'permalink': 'myheritage'}}},
 {'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'blogT

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [34]:

query = {}
projection = {"name":1, "founded_year":1,"_id":0}
list(c.find(query, projection).sort("founded_year"))[:5]

[{'name': 'Flektor', 'founded_year': None},
 {'name': 'Lala', 'founded_year': None},
 {'name': 'SpinVox', 'founded_year': None},
 {'name': 'Gannett', 'founded_year': None},
 {'name': 'Info', 'founded_year': None}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [19]:
condition_1 =  {"founded_day":{"$lte":7}}
query = condition_1
projection = {"name":1,"_id":0}

list(c.find(query, projection).sort("acquisition.price_amount", -1).limit(10))

[{'name': 'Netscape'},
 {'name': 'PayPal'},
 {'name': 'Zappos'},
 {'name': 'Alibaba'},
 {'name': 'Postini'},
 {'name': 'Danger'},
 {'name': 'Clearwell Systems'},
 {'name': 'PrimeSense'},
 {'name': 'Amobee'},
 {'name': 'Vitrue'}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [20]:
condition_1 =  {"number_of_employees":{"$gt":4000}}
condition_2= {"category_code":"web"}
query={"$and": [condition_1, condition_2]}
projection = {"name":1,"_id":0}

list(c.find(query, projection).sort("number_of_employees"))

[{'name': 'Expedia'},
 {'name': 'AOL'},
 {'name': 'Webkinz'},
 {'name': 'Rakuten'},
 {'name': 'Los Angeles Times Media Group'},
 {'name': 'Groupon'},
 {'name': 'Yahoo!'},
 {'name': 'eBay'},
 {'name': 'Experian'}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [21]:
condition_1 =  {"acquisition.price_amount":{"$gt":10000000}}
condition_2= {"acquisition.price_currency_code":"EUR"}
query={"$and": [condition_1, condition_2]}
projection = {"name":1,"_id":0}

list(c.find(query, projection))

[{'name': 'ZYB'},
 {'name': 'Apertio'},
 {'name': 'Greenfield Online'},
 {'name': 'Webedia'},
 {'name': 'Wayfinder'},
 {'name': 'Tuenti Technologies'},
 {'name': 'BioMed Central'}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [22]:
condition_1 =  {"acquisition.acquired_month":{"$lte":3}}
query = condition_1
projection = {"name":1,"acquisition":1, "_id":0}

list(c.find(query, projection).limit(10))

[{'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'NetRatings',
  'acquisition': {'price_amount': 327000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L',
   'source_description': 'Nielsen buys rest of NetRatings',
   'acquired_year': 2007,
   'acquired_month': 2,
   

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [23]:
condition_1={"founded_year":{"$lte":2010}}
condition_1={"founded_year":{"$gte":2000}}
condition_3 = {"acquisition.acquired_year":{"$gte":2011}}
query={"$and":[condition_1, condition_2, condition_3]}
projection={"name":1, "_id":0}
list(c.find(query, projection))

[{'name': 'Citizenside'}, {'name': 'Webedia'}, {'name': 'ulike'}]

### 20. All the companies that have been 'deadpooled' after the third year.

In [37]:
condition_1={"deadpooled_year":{"$gt":3}}
query=condition_1
projection={"name":1, "_id":0}
list(c.find(query, projection))[:5]

[{'name': 'Omnidrive'},
 {'name': 'Babelgum'},
 {'name': 'Sparter'},
 {'name': 'Thoof'},
 {'name': 'Mercora'}]

**⚠️ Did you do this?**

```python
list(collection.find(query))[:5] #or a reasonably low number
```